# Pycosat implementation of Algo #1

In [2]:
import pycosat

In [3]:
# Solver for Numberlink
class NumberLinkSolver:
    pass

In [4]:
# choose variables
# we need a number per edge
# each edge is ((x1, y1), (x2, y2))
# x_{edge, index of path}
from typing import Optional


def x(self, edge, index):
    def edge_to_num(edge):
        v, w = edge
        x1, y1 = v
        x2, y2 = w
        return x1 * self.n**3 + y1 * self.n**2 + x2 * self.n + y2

    num_paths = self.num_paths
    ans = edge_to_num(edge) * num_paths + index
    # +1 because we start from
    return ans + 1


NumberLinkSolver.x = x


def inverse_x(self, var):
    n = self.n
    num_paths = self.num_paths

    var -= 1
    e, index = divmod(var, num_paths)
    rest, y2 = divmod(e, n)
    rest, x2 = divmod(rest, n)
    x1, y1 = divmod(rest, n)

    return ((x1, y1), (x2, y2)), index


NumberLinkSolver.inverse_x = inverse_x

In [5]:
# HELPER METHODS
def all(literals):
    return [[literal] for literal in literals]


def none(literals):
    return [[-literal] for literal in literals]


def at_least_one(literals):
    return [literals]


def at_most_one(literals):
    return [[-l1, -l2] for l1 in literals for l2 in literals if l1 < l2]


def exactly_one(literals):
    return at_least_one(literals) + at_most_one(literals)

In [6]:
def at_most_one_path_per_edge(self):
    num_paths = self.num_paths
    edges = self.edges
    x = self.x

    clauses = []
    for e in edges:
        literals = [x(e, i) for i in range(num_paths)]
        clauses += at_most_one(literals)
    return clauses


NumberLinkSolver.at_most_one_path_per_edge = at_most_one_path_per_edge

In [7]:
def exactly_one_edge_per_vertex_on_path(self):
    num_paths = self.num_paths
    pairs = self.pairs
    x = self.x
    vertex_to_edges = self.vertex_to_edges

    clauses = []
    for index in range(num_paths):
        vertices = pairs[index]
        for v in vertices:
            literals = [x(e, index) for e in vertex_to_edges[v]]
            clauses += exactly_one(literals)

            # don't want these vertices to be on any other non-index paths
            for i in range(num_paths):
                if i != index:
                    literals = [x(e, i) for e in vertex_to_edges[v]]
                    clauses += none(literals)
    return clauses


NumberLinkSolver.exactly_one_edge_per_vertex_on_path = (
    exactly_one_edge_per_vertex_on_path
)

In [ ]:
• each vertex v ∈ V not in a pair of X is contained in precisely two edges of one of the paths.

def exactly_two_edges_per_vertex_not_in_pair(self):
    num_paths = self.num_paths
    pairs = self.pairs
    x = self.x
    vertex_to_edges = self.vertex_to_edges
    vertices = self.vertices

    # for each vertex, we require at least one edge among all paths to be active
    # for each edge incident on vertex in path i, if it is active, then at least one of the other edges incident on the vertex in path i must be active
    # \forall e \in E
    # e => 

    pair_vertices = {v for pair in pairs for v in pair}
    clauses = []
    for v in vertices:
        if v in pair_vertices:
            continue

        for index in range(num_paths):
            literals = [x(e, index) for e in vertex_to_edges[v]]

            

        literals = [x(e, i) for e in vertex_to_edges[v] for i in range(num_paths)]
        clauses += exactly_one(literals)
    return clauses

In [ ]:
from collections import defaultdict

def __init__(self, puzzle):



NumberLinkSolver.__init__ = __init__

In [15]:
def solve(self):
    clauses = []
    clauses += self.each_vertex_in_a_single_path()
    clauses += self.each_position_in_a_single_vertex()
    clauses += self.path_finished()
    clauses += self.consecutive_vertices_along_path()
    clauses += self.source_and_sink()

    literals = pycosat.solve(clauses)

    if literals == "UNSAT":
        return None

    n = self.n
    solution = [[0 for _ in range(n)] for _ in range(n)]
    for literal in literals:
        if literal > 0:
            vertex, index, _ = self.inverse_x(literal)
            if vertex is None:
                continue
            x, y = vertex
            solution[x][y] = index + 1

    return solution


NumberLinkSolver.solve = solve

In [16]:
puzzle = [[1, 0, 1], [2, 0, 2], [3, 0, 3]]

solver = NumberLinkSolver(puzzle)
print(solver.solve())  # [[1, 1, 1], [2, 2, 2], [3, 3, 3]]

[[1, 1, 1], [2, 2, 2], [3, 3, 3]]
